# Fotballanalyse Setup

Dette noteboket vil hjelpe oss med å:
1. Analysere prosjektstrukturen
2. Undersøke CSV-dataene
3. Planlegge feature engineering
4. Definere treningsstrategien

## 1. Prosjektstruktur

Først skal vi se på hvilke filer vi har i prosjektet:

In [2]:
import os

# List opp filer i prosjektet
project_files = ['Index.html', 'README.md', 'data_scrapping.ipynb', 'main.js', 
                'requirements.txt', 'seriea_data_analyse.ipynb', 'styles.css', 
                'setup_analysis.ipynb']
project_dirs = ['data', 'models', 'src']

print("Filer i prosjektet:")
for f in project_files:
    print(f"- {f}")

print("\nMapper i prosjektet:")
for d in project_dirs:
    print(f"- {d}/")

Filer i prosjektet:
- Index.html
- README.md
- data_scrapping.ipynb
- main.js
- requirements.txt
- seriea_data_analyse.ipynb
- styles.css
- setup_analysis.ipynb

Mapper i prosjektet:
- data/
- models/
- src/


## 2. Data Analyse

La oss undersøke CSV-filene og se hvilken som er best å bruke for trening:

In [2]:
import pandas as pd
from io import StringIO
import os

def read_file_content(file_path):
    with open(file_path, 'r') as f:
        return f.read()

# Example content from serie_a_data_2020-2026.csv
example_content = '''rk,squad,mp,w,d,l,gf,ga,gd,pts,pts/mp,xg,xga,xgd,xgd/90,attendance,top_team_scorer,goalkeeper,notes,sesonger,last_5
1,Milan,38,26,8,4,69,31,38,86,2.26,59.7,35.9,23.8,0.63,44015,"Olivier Giroud, Rafael Leão - 11",Mike Maignan,→ Champions League via league finish,2020,
2,Inter,38,25,9,4,84,32,52,84,2.21,81.6,39.1,42.5,1.12,44473,Lautaro Martínez - 21,Samir Handanović,→ Champions League via league finish,2020,
3,Napoli,38,24,7,7,74,31,43,79,2.08,57.8,32.5,25.3,0.67,28119,Victor Osimhen - 14,David Ospina,→ Champions League via league finish,2020,'''

df = pd.read_csv(StringIO(example_content))
print("Analyse av eksempeldata fra Serie A 2020-2026:")
print(f"Antall rader: {len(df)}")
print(f"\nKolonner: {df.columns.tolist()}")
print("\nFørste 3 rader:")
print(df.head(3))
print("\nDatatyper:")
print(df.dtypes)
print("\nBeskrivende statistikk:")
print(df.describe())

Analyse av eksempeldata fra Serie A 2020-2026:
Antall rader: 3

Kolonner: ['rk', 'squad', 'mp', 'w', 'd', 'l', 'gf', 'ga', 'gd', 'pts', 'pts/mp', 'xg', 'xga', 'xgd', 'xgd/90', 'attendance', 'top_team_scorer', 'goalkeeper', 'notes', 'sesonger', 'last_5']

Første 3 rader:
   rk   squad  mp   w  d  l  gf  ga  gd  pts  ...    xg   xga   xgd  xgd/90  \
0   1   Milan  38  26  8  4  69  31  38   86  ...  59.7  35.9  23.8    0.63   
1   2   Inter  38  25  9  4  84  32  52   84  ...  81.6  39.1  42.5    1.12   
2   3  Napoli  38  24  7  7  74  31  43   79  ...  57.8  32.5  25.3    0.67   

   attendance                   top_team_scorer        goalkeeper  \
0       44015  Olivier Giroud, Rafael Leão - 11      Mike Maignan   
1       44473             Lautaro Martínez - 21  Samir Handanović   
2       28119               Victor Osimhen - 14      David Ospina   

                                  notes sesonger  last_5  
0  → Champions League via league finish     2020     NaN  
1  → Champions Le

## 3. Feature Engineering Plan

Basert på tilgjengelige data fra Serie A, kan vi lage følgende features:

1. Prestasjonsmetrikker:
   - Poeng per kamp (pts/mp)
   - Expected Goals (xG) og Expected Goals Against (xGA)
   - Målforskjell per kamp (gd/mp)
   - xG-differanse per 90 minutter (xgd/90)

2. Form og momentum:
   - Siste 5 kampers resultater
   - Hjemme/borte form (basert på w/d/l)
   - Gjennomsnittlig målscoring siste N kamper
   - Gjennomsnittlig xG siste N kamper

3. Lagets styrke:
   - Keeper kvalitet (clean sheets, redninger)
   - Toppscorer statistikk
   - Publikumsstøtte (attendance)
   - Tabellposisjon (rk)

4. Sesongkontekst:
   - Hvilket år i datasettet (sesonger)
   - Gjenstående kamper i sesongen
   - Kamp om Champions League/Europa League plass (basert på notes)

5. Avanserte metrikker:
   - Forholdet mellom faktiske mål og xG
   - Defensiv effektivitet (ga/xga)
   - Offensiv effektivitet (gf/xg)

## 4. Treningsstrategi

For Serie A-prediksjoner, bruker vi følgende strategi:

1. Dataprosessering:
   - Splitte data i trenings- (2020-2024) og testsett (2024-2026)
   - Normalisere numeriske features (spesielt xG, attendance, pts)
   - Encode kategoriske variabler (lag, keeper, toppscorer)
   - Håndtere manglende verdier i last_5 og attendance

2. Modellvalg:
   - Baseline: Logistisk regresjon med:
     * Win/Draw/Loss prediksjon
     * Mål-intervall prediksjon (0,1,2,3,4+)
   - Avanserte modeller:
     * XGBoost (god for tabellariske data)
     * LightGBM (effektiv med kategoriske variabler)
     * Deep Learning for komplekse mønstre

3. Evaluering:
   - Tidsserie cross-validation (hver sesong som en fold)
   - Metrics:
     * Accuracy for kampresultat
     * Mean Absolute Error for målprediksjon
     * Log-loss for sannsynlighetsestimater
     * ROC-AUC for binære utfall

4. Hyperparameter Tuning:
   - Bayesian Optimization for effektiv søk
   - Separate modeller for:
     * Kampresultat (W/D/L)
     * Totalt antall mål
     * Over/Under 2.5 mål

5. Produksjonssetting:
   - API for prediksjon av kommende kamper
   - Periodisk retraining på nye data
   - Monitorering av modellytelse over tid